In [1]:
!pip -q install langchain openai huggingface_hub transformers tiktoken faiss-cpu flask flask_ngrok pyngrok==4.1.1

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!ngrok authtoken 2RA0tLTxHtqWNZ91w7oVm8fWzcV_2DDirNN4J16413H6iioso

                                                                                                    
Installing ngrok ... 
                                                                                                    
Authtoken saved to configuration file: C:\Users\Admin/.ngrok2/ngrok.yml


In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, session

app = Flask(__name__,static_folder='static')
run_with_ngrok(app)
app.secret_key = 'your-secret-key'

# Setup app
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI




qa_chain = None

def initialize_qa_chain(link, api_key):
    global qa_chain
    # Initialize the QA chain with the provided link and API key
    os.environ['OPENAI_API_KEY'] = api_key
    # clone repo
    os.system(f"git clone {link}")

    embeddings = OpenAIEmbeddings()

    # Set up db
    root_dir = get_folder_name(link)
    texts = []

    # Go through each folder
    for dirpath, dirnames, filenames in os.walk(root_dir):

        # Go through each file
        for file in filenames:
            try:
                # Load up the file as a doc and split
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                texts.extend(loader.load_and_split())
            except Exception as e:
                pass

    docsearch = FAISS.from_documents(texts, embeddings)
    retriever=docsearch.as_retriever()
    # retriever.search_kwargs['distance_metric'] = 'cos'
    retriever.search_kwargs['k'] = 4

    qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)


@app.route('/', methods=['GET', 'POST'])
def home():

    initialized = False
    if request.method == 'POST':
        if 'link' in request.form and 'api_key' in request.form:
            link = request.form['link']
            api_key = request.form['api_key']
            initialize_qa_chain(link, api_key)
            initialized = True
        elif 'question' in request.form:
            question = request.form['question']
            result = process_question(question)
            valid_qa_chain = True
            if result == "":
                result = "Please initialize repo before querying"
                valid_qa_chain = False
            return render_template('index.html', result=result, initialized=True&valid_qa_chain)
    return render_template('index.html', initialized=initialized)

def process_question(question):
    if qa_chain:
      ans = qa_chain({"query": question})
      res = ans["result"]
      response = f"Response = {res}"
    else:
      response = ""
    return response

def get_folder_name(repo_link):
  if repo_link.endswith('.git'):
        repo_link = repo_link[:-4]

  # Split the repository link by the forward slash '/'
  split_link = repo_link.split('/')

  # Retrieve the last part of the split link, which is the repository folder name
  folder_name = split_link[-1]

  return folder_name

if __name__ == '__main__':
    app.run()


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)